In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/json-nav-cleaned/nav_new.json
/kaggle/input/json-nav-cleaned/meta.json
/kaggle/input/nav-data-cleaned/nav_new.csv


The goal here will be to create a model based on Daily NAV values, Daily Returns and Ranking of those Daily NAV.  
The model will predict the future ranking of the scheme. This should help you to pick the best scheme available.

In [2]:
nav_csv = pd.read_csv("/kaggle/input/nav-data-cleaned/nav_new.csv", index_col = 0 )

In [3]:
change = nav_csv.pct_change().add_suffix("_1Day")

In [4]:
rank = change.rank(axis = 1, method="max").add_suffix("Rank")

In [5]:
nav_csv

,102652,102653,102654,102645,102644,102646,102647,102649,102648,102650,...,151799,151797,151898,151895,151896,151897,151865,151863,151862,151864
2006-04-03 00:00:00,11.7436,10.5045,10.4490,10.0,10.8580,10.0078,0.0,0.0,10.2871,0.0,...,10.0454,10.0454,10.016,10.017,10.017,10.016,10.0091,10.0091,10.0091,10.0091
2006-04-04 00:00:00,11.7380,10.4995,10.4440,10.0,10.8598,10.0095,0.0,0.0,10.2889,0.0,...,10.0075,10.0075,10.018,10.019,10.019,10.018,10.0119,10.0128,10.0115,10.0131
2006-04-05 00:00:00,11.7663,10.5249,10.4692,10.0,10.8633,10.0127,0.0,0.0,10.2925,0.0,...,9.9849,9.9849,10.020,10.022,10.022,10.020,10.0189,10.0198,10.0185,10.0201
2006-04-06 00:00:00,11.7392,10.5006,10.4451,10.0,10.8633,10.0127,0.0,0.0,10.2925,0.0,...,10.0201,10.0201,10.024,10.027,10.027,10.024,10.0265,10.0274,10.0261,10.0277
2006-04-07 00:00:00,11.7944,10.5500,10.4942,10.0,10.8650,10.0142,0.0,10.0,10.2941,0.0,...,10.0041,10.0041,10.026,10.029,10.029,10.026,10.0284,10.0294,10.0280,10.0296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
change

,102652_1Day,102653_1Day,102654_1Day,102645_1Day,102644_1Day,102646_1Day,102647_1Day,102649_1Day,102648_1Day,102650_1Day,...,151799_1Day,151797_1Day,151898_1Day,151895_1Day,151896_1Day,151897_1Day,151865_1Day,151863_1Day,151862_1Day,151864_1Day
2006-04-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-04-04 00:00:00,-0.000477,-0.000476,-0.000479,0.0,0.000166,0.00017,NaN,NaN,0.000175,NaN,...,-0.003773,-0.003773,0.000200,0.000200,0.000200,0.000200,0.000280,0.000370,0.000240,0.000400
2006-04-05 00:00:00,0.002411,0.002419,0.002413,0.0,0.000322,0.00032,NaN,NaN,0.000350,NaN,...,-0.002258,-0.002258,0.000200,0.000299,0.000299,0.000200,0.000699,0.000699,0.000699,0.000699
2006-04-06 00:00:00,-0.002303,-0.002309,-0.002302,0.0,0.000000,0.00000,NaN,NaN,0.000000,NaN,...,0.003525,0.003525,0.000399,0.000499,0.000499,0.000399,0.000759,0.000758,0.000759,0.000758
2006-04-07 00:00:00,0.004702,0.004704,0.004701,0.0,0.000156,0.00015,NaN,inf,0.000155,NaN,...,-0.001597,-0.001597,0.000200,0.000199,0.000199,0.000200,0.000189,0.000199,0.000190,0.000189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27 00:00:00,0.000000,0.000000,0.000000,0.0,NaN,0.00000,NaN,0.0,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-08-28 00:00:00,0.000000,0.000000,0.000000,0.0,NaN,0.00000,NaN,0.0,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-08-29 00:00:00,0.000000,0.000000,0.000000,0.0,NaN,0.00000,NaN,0.0,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-08-30 00:00:00,0.000000,0.000000,0.000000,0.0,NaN,0.00000,NaN,0.0,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
rank

,102652_1DayRank,102653_1DayRank,102654_1DayRank,102645_1DayRank,102644_1DayRank,102646_1DayRank,102647_1DayRank,102649_1DayRank,102648_1DayRank,102650_1DayRank,...,151799_1DayRank,151797_1DayRank,151898_1DayRank,151895_1DayRank,151896_1DayRank,151897_1DayRank,151865_1DayRank,151863_1DayRank,151862_1DayRank,151864_1DayRank
2006-04-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-04-04 00:00:00,3230.0,3231.0,3227.0,7031.0,10843.0,11104.0,NaN,NaN,11449.0,NaN,...,921.0,921.0,12911.0,12890.0,12890.0,12911.0,18644.0,20838.0,16234.0,21412.0
2006-04-05 00:00:00,31957.0,31963.0,31960.0,7231.0,19462.0,19414.0,NaN,NaN,19929.0,NaN,...,1492.0,1492.0,12402.0,18823.0,18823.0,12402.0,25281.0,25278.0,25283.0,25277.0
2006-04-06 00:00:00,2259.0,2258.0,2260.0,8339.0,8339.0,8339.0,NaN,NaN,8339.0,NaN,...,33225.0,33225.0,21345.0,22970.0,22970.0,21345.0,26779.0,26776.0,26787.0,26775.0
2006-04-07 00:00:00,33462.0,33464.0,33461.0,8028.0,11004.0,10882.0,NaN,34901.0,10992.0,NaN,...,2240.0,2240.0,12982.0,12933.0,12933.0,12982.0,12461.0,12927.0,12462.0,12454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27 00:00:00,33531.0,33531.0,33531.0,33531.0,NaN,33531.0,NaN,33531.0,33531.0,NaN,...,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0
2023-08-28 00:00:00,33530.0,33530.0,33530.0,33530.0,NaN,33530.0,NaN,33530.0,33530.0,NaN,...,33530.0,33530.0,33530.0,33530.0,33530.0,33530.0,33530.0,33530.0,33530.0,33530.0
2023-08-29 00:00:00,33531.0,33531.0,33531.0,33531.0,NaN,33531.0,NaN,33531.0,33531.0,NaN,...,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0,33531.0
2023-08-30 00:00:00,33537.0,33537.0,33537.0,33537.0,NaN,33537.0,NaN,33537.0,33537.0,NaN,...,33537.0,33537.0,33537.0,33537.0,33537.0,33537.0,33537.0,33537.0,33537.0,33537.0


In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM